In [13]:
import torch
from typing import List
import pandas as pd
from collections import defaultdict
import bittensor as bt
import requests
import plotly.graph_objects as go


This notebook demonstrates the behavior of the validator and how rewards are calculated.
Additianlly it shows how miners are rated and scored in a plot.

In [14]:
# NASChain NetUID
naschain_uid = 31
subtensor = bt.subtensor()
metagraph = subtensor.metagraph(naschain_uid)
genomaster_ip = 'http://51.161.12.128'
genomaster_port = '5000'
genomaster_valid_endpoint= '/handle_valid_request'

2024-04-10 03:26:50.624 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-04-10 03:26:50.625 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-04-10 03:26:50.625 |     WARNING      | In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it.
2024-04-10 03:26:51.124 |       INFO       | Connected to finney network and wss://entrypoint-finney.opentensor.ai:443.


In [15]:
def reward(df: pd.DataFrame , tolerance : int):
    # Initialize the dictionary to store user scores and job counts
    user_scores = defaultdict(lambda: {'points': 0, 'accepted_jobs': 0, 'rejected_jobs': 0})

    # Process each job batch
    for _, group in df.groupby('Genome_String'):
        responses = group['Response'].tolist()
        user_ids = group['Assigned_User'].tolist()

        # Initialize agreement checks
        agreements = [False] * len(user_ids)  # Default all to False

        # Check pairwise agreement within tolerance for the first element and exact match for others
        for i in range(len(responses)):
            for j in range(i + 1, len(responses)):
                # Check agreement between i and j
                agree_first = abs(responses[i][0] - responses[j][0]) <= tolerance
                agree_second = responses[i][1] == responses[j][1]
                agree_third = responses[i][2] == responses[j][2]

                # Update agreement status
                if agree_first and agree_second and agree_third:
                    agreements[i] = True
                    agreements[j] = True

        # Update scores based on agreement
        for i, user_id in enumerate(user_ids):
            if agreements[i]:
                user_scores[user_id]['points'] += 1
                user_scores[user_id]['accepted_jobs'] += 1
            else:
                user_scores[user_id]['rejected_jobs'] += 1

    # Normalize the points and prepare the lists
    total_points = sum(user['points'] for user in user_scores.values())
    normalized_scores_list = []
    user_ids_list = []

    # Fill the lists with normalized scores and user IDs
    for user_id, score in user_scores.items():
        normalized_score = score['points'] / total_points if total_points > 0 else 0
        normalized_scores_list.append(normalized_score)
        user_ids_list.append(user_id)

    
    
    all_users_in_metagraoh = list(range(int(metagraph.n)))
    all_scores_tensor = torch.zeros(int(metagraph.n))
    # Scatter the scores to the corresponding user IDs
    all_scores_tensor[user_ids_list] = torch.tensor(normalized_scores_list)
    return all_scores_tensor, all_users_in_metagraoh, user_scores

def plot_sorted_rewards(uids, rewards):
 # Combine uids and rewards into a single list of tuples and sort by rewards in descending order
    combined = list(zip(uids, rewards))
    combined_sorted = sorted(combined, key=lambda x: x[1], reverse=True)

    # Unzip the sorted combined list back into uids and rewards
    sorted_uids, sorted_rewards = zip(*combined_sorted)

    # Create the bar chart with Plotly, ensuring it's correctly sorted
    fig = go.Figure(go.Bar(
        x=sorted_uids,
        y=sorted_rewards,
        marker_color='rgb(55, 83, 109)'  # Set bar color
    ))

    fig.update_layout(
        title='UIDs Sorted by Rewards (Corrected)',
        xaxis_title='UID',
        yaxis_title='Reward',
        template='plotly_white'
    )

    return fig


In [16]:
# Establish a connection to GenoMaster to request all the jobs that
# have been completed in the current Genetic Algorithm population.
try:
        # Construct the full URL
        url = genomaster_ip + ':' + genomaster_port + '/handle_valid_request'
        response = requests.get(url)  # This is a synchronous call, consider using an async library if needed
        # Check the status code of the response
        if response.status_code == 200:
            bt.logging.info("✅ Validation request successful.")
            # Convert the JSON response to a pandas DataFrame
            response_data = response.json()
            df = pd.DataFrame(response_data)
            # check to seee if there is new batch of data for validation is ready in genomaster 
            rewards, uids, msgs = reward(df,tolerance=1)
            print("uids", uids)
            print("rewards", rewards)
            fig = plot_sorted_rewards(uids, rewards)
            fig.show()

        
        elif response.status_code == 503:
            bt.logging.warning("⏳ GenoMaster is currently busy, connections will be accepted soon.")

        else:
            # bt.logging.error("❌ Validation request failed.")
            # bt.logging.error("❌ Status code:", response.status_code)
            bt.logging.warning("⏳ Validation request not complete. Message:", response.json().get('message'))

except Exception as e:
        bt.logging.error(f"❌ An error occurred: {e}")
        # Optionally, print the traceback if you need more details


2024-04-10 03:26:58.970 |       INFO       | ✅ Validation request successful.
uids [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58]
rewards tensor([0.0653, 0.0536, 0.0326, 0.0490, 0.0000, 0.0676, 0.0350, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0769, 0.0000, 0.0000, 0.0000, 0.0536, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0746, 0.0000, 0.0000, 0.0000, 0.0256, 0.0000,
        0.0117, 0.0000, 0.0350, 0.0373, 0.0000, 0.0443, 0.0256, 0.0373, 0.0326,
        0.0373, 0.0326, 0.0350, 0.0350, 0.0000, 0.0000, 0.0000, 0.0303, 0.0000,
        0.0396, 0.0000, 0.0326, 0.0000, 0.0000])
